ΒΗΜΑ 1 

Εγκατάσταση βιβλιοθηκών

In [5]:
!pip install requests beautifulsoup4

Import βιβλιοθηκών

In [6]:
import requests
from bs4 import BeautifulSoup
import json

Η παρακάτω συνάρτηση αναζητά άρθρα απο το wikipedia, έχοντας ως είσοδο ένα searchQuery και τον αριθμό των άρθρων που θελουμε να λάβουμε. Η συνάρτηση αυτη μόλις εκτελεστεί θα επιστρέψει το περιεχόμενο των άρθρων που βρέθηκαν και τα αποθηκεύει σε ένα αρχείο json.

In [7]:
def scrape_wikipedia(searchQuery, max_articles=5):
    
    base_url = "https://en.wikipedia.org"
    search_url = f"{base_url}/w/index.php?search={searchQuery}&title=Special:Search&fulltext=1"

    print(f"Αναζήτηση για άρθρα στο Wikipedia σχετικά με: '{searchQuery}'...")

    response = requests.get(search_url)
    if response.status_code != 200:
        print("Δεν ήταν δυνατή η ανάκτηση της σελίδας αναζήτησης. Παρακαλώ δοκιμάστε ξανά.")
        return []

    bSoup = BeautifulSoup(response.text, "html.parser")
    articleLinks = []

    search_results = bSoup.select(".mw-search-result-heading a")
    for link in search_results:
        articleLinks.append(base_url + link["href"])
        if len(articleLinks) >= max_articles:
            break

    print(f"Βρέθηκαν {len(articleLinks)} σύνδεσμοι άρθρων. Λήψη εγγράφων...")

    articles = []

    for i, url in enumerate(articleLinks):
        print(f"Λήψη άρθρου {i + 1}: {url}")

        response = requests.get(url)
        if response.status_code != 200:
            print(f"Αποτυχία λήψης του άρθρου {i + 1}. Παράβλεψη...")
            continue

        articleSoup = BeautifulSoup(response.text, "html.parser")

        articleTitle = articleSoup.find("h1", id="firstHeading")
        if  articleTitle:
            title = articleTitle.text
        else:
            title = "Χωρίς τίτλο"

        para = articleSoup.find_all("p")
        articleContent = ""  
        for p in para:
            articleContent += p.text + " "

        articles.append({
            "title": title,
            "url": url,
            "content": articleContent
        })

    fileName = searchQuery.replace(" ", "_") + "_wikipedia.json"
    with open(fileName, "w", encoding="utf-8") as file:
        json.dump(articles, file, indent=4, ensure_ascii=False)

    print(f"Τα άρθρα αποθηκεύτηκαν στο αρχείο '{fileName}'")

#---------Εκτέλεση της παραπάνω συνάρτησης-----------------------#

searchQuery = input("Εισάγετε τo keyword για αναζήτηση άρθρων στο Wikipedia: ")
maxArticles = int(input("Εισάγετε το μέγιστο πλήθος άρθρων προς λήψη: "))
scrape_wikipedia(searchQuery, maxArticles)

Αναζήτηση για άρθρα στο Wikipedia σχετικά με: 'machine learning'...
Βρέθηκαν 15 σύνδεσμοι άρθρων. Λήψη εγγράφων...
Λήψη άρθρου 1: https://en.wikipedia.org/wiki/Machine_learning
Λήψη άρθρου 2: https://en.wikipedia.org/wiki/Quantum_machine_learning
Λήψη άρθρου 3: https://en.wikipedia.org/wiki/Neural_network_(machine_learning)
Λήψη άρθρου 4: https://en.wikipedia.org/wiki/Attention_(machine_learning)
Λήψη άρθρου 5: https://en.wikipedia.org/wiki/Active_learning_(machine_learning)
Λήψη άρθρου 6: https://en.wikipedia.org/wiki/Boosting_(machine_learning)
Λήψη άρθρου 7: https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)
Λήψη άρθρου 8: https://en.wikipedia.org/wiki/Adversarial_machine_learning
Λήψη άρθρου 9: https://en.wikipedia.org/wiki/Timeline_of_machine_learning
Λήψη άρθρου 10: https://en.wikipedia.org/wiki/Automated_machine_learning
Λήψη άρθρου 11: https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)
Λήψη άρθρου 12: https://en.wikipedia.org/wiki/Artificial_int

ΒΗΜΑ 2

Εγκατάσταση της βιβλιοθήκης nltk

In [8]:
!pip install nltk

Import βιβλιοθηκών

In [9]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\klodi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\klodi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\klodi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True


Η συνάρτηση που ακολουθεί καλείτε απο την επόμενη συνάρτηση, για την επεξεργασία του αρχείου json. Συγκεκριμένα:
    1. Μετατρέπει το κείμενο σε πεζά γράμματα.
    2. Αφαιρεί όλους τους χαρακτήρες εκτός από γράμματα και κενά.
    3. Διαχωρίζει το κείμενο σε λέξεις.
    4. Απορρίπτει τα stop words όπως the, and, κ.λπ.
    5. Εφαρμόζει lemmatization για να μετατρέψει τις λέξεις στη βασική τους μορφή (π.χ., running → run).
Τέλος, η συνάρτηση επιστρέφει μία λίστα, η οποία περιέχει όλες τις επεξεργασμένες λέξεις.

In [10]:
from nltk.stem import WordNetLemmatizer
import re

def processedText(words):
   
    words = words.lower()
    words = re.sub(r"[^a-zA-Z\s]", " ", words)  
    tokens = word_tokenize(words)
    stopWords = set(stopwords.words('english'))  
    tokens = [word for word in tokens if word not in stopWords]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

#---------------Παράδειγμα εκτέλεσης της συνάρτησης------------------#

def processedText_test():
   sentences = [
       "The quick brown fox jumps over the lazy dog!",
       "Sarah's birthday party was amazing & fun!",
       "Learning Python programming is VERY interesting.",
       "The cat's whiskers twitched while it was sleeping...",
       "Will you be attending the meeting tomorrow?"
   ]
   
   for sentence in sentences:
       print("\nΑρχική πρόταση:", sentence)
       print("Επεξεργασμένη πρόταση:", processedText(sentence))

processedText_test()



Αρχική πρόταση: The quick brown fox jumps over the lazy dog!
Επεξεργασμένη πρόταση: ['quick', 'brown', 'fox', 'jump', 'lazy', 'dog']

Αρχική πρόταση: Sarah's birthday party was amazing & fun!
Επεξεργασμένη πρόταση: ['sarah', 'birthday', 'party', 'amazing', 'fun']

Αρχική πρόταση: Learning Python programming is VERY interesting.
Επεξεργασμένη πρόταση: ['learning', 'python', 'programming', 'interesting']

Αρχική πρόταση: The cat's whiskers twitched while it was sleeping...
Επεξεργασμένη πρόταση: ['cat', 'whisker', 'twitched', 'sleeping']

Αρχική πρόταση: Will you be attending the meeting tomorrow?
Επεξεργασμένη πρόταση: ['attending', 'meeting', 'tomorrow']


Στόχος αυτής της συνάρτησης είναι να λάβει ως είσοδο το αρχείο json και για κάθε άρθρο να καλείται η παραπάνω συνάρτηση για την επεξεργασία του. Τέλος, τα αποτελέσματα που επιστρεφεί η processedText 
αποθηκεύονται σε ενά νέο αρχείο json, όπου πλεόν θα περιέχει όλα τα άρθρα επεξεργασμένα.  

In [11]:
def processDataset(searchQuery):

    searchQuery = searchQuery.replace(" ", "_").lower()
    inFile = f"{searchQuery}_wikipedia.json"
    outFile = "processed_" + inFile

    with open(inFile, "r", encoding="utf-8") as file:
        data = json.load(file)
    
    processedData = []
    
    for article in data:
        title = article.get("title", "")
        content = article.get("content", "")
        
        processedTitle = processedText(title)
        processedContent = processedText(content)
        
        processedData.append({
            "title": " ".join(processedTitle),
            "content": " ".join(processedContent),
            "url": article.get("url", "")
        })
    
    with open(outFile, "w", encoding="utf-8") as file:
        json.dump(processedData, file, indent=4, ensure_ascii=False)

    print(f"Το 'καθαρό' σύνολο δεδομένων αποθηκεύτηκε στο αρχείο: {outFile}")

processDataset(searchQuery)


Το 'καθαρό' σύνολο δεδομένων αποθηκεύτηκε στο αρχείο: processed_machine_learning_wikipedia.json


ΒΗΜΑ 3

Η συναρτηση λαμβάνει το json αρχείο που έχει παραχθεί παραπάνω και σκοπός της είναι να επιστρέψει ένα αντεστραμμένο ευρετήριο σε μορφή json. Το ευρετήριο αυτό περίεχει κάθε λέξη που εμφανίζεται στα άρθρα και πιο συγκεκριμενα εμφανιζει σε ποια άρθρα βρισκεται αυτη η λεξη , καθώς και την συχνότητα εμφάνισής της. Το τελικό ευρετήριο αποθηκεύεται σε νέο αρχείο inverted_index.json, παρέχοντας μια αποδοτική δομή για αναζητήσεις λέξεων.

In [12]:
def invertedIndex(searchQuery):
   
    searchQuery = searchQuery.replace(" ", "_").lower()
    inFile = f"processed_{searchQuery}_wikipedia.json"
    outFile = "inverted_index.json"  

    try:
        with open(inFile, "r", encoding="utf-8") as file:
            data = json.load(file)
        print(f"Διαβάστηκαν {len(data)} άρθρα από το αρχείο {inFile}.")
        
        invertIndex = {}  

        for doc_id, article in enumerate(data, start=1):
            content = article.get('content', '')
            title = article.get('title', '')
            text = title + " " + content
            
            words = text.split() 
            
            for word in words:
                if word not in invertIndex:
                    invertIndex[word] = {"documents": [], "frequency": {}}
                
                if doc_id not in invertIndex[word]["documents"]:
                    invertIndex[word]["documents"].append(doc_id)
                    invertIndex[word]["frequency"][doc_id] = 0  
                
                invertIndex[word]["frequency"][doc_id] += 1
        
        with open(outFile, "w", encoding="utf-8") as out_file:
            json.dump(invertIndex, out_file, indent=4, ensure_ascii=False)
        print(f"Το αντεστραμμένο ευρετήριο αποθηκεύτηκε στο αρχείο {outFile}.")
        
    except FileNotFoundError:
        print("Το αρχείο δεδομένων δεν βρέθηκε. Βεβαιωθείτε ότι υπάρχει το αρχείο JSON.")


invertedIndex(searchQuery)

Διαβάστηκαν 15 άρθρα από το αρχείο processed_machine_learning_wikipedia.json.
Το αντεστραμμένο ευρετήριο αποθηκεύτηκε στο αρχείο inverted_index.json.


ΒΗΜΑ 4

Ερώτημα α)

Η συνάρτηση loadIndex φορτώνει το αντεστραμμένο ευρετήριο από ένα JSON αρχείο. Αρχικά, παίρνει το όνομα του αρχείου και στη συνέχεια προσπαθεί να το διαβάσει και να επιστρέψει τα δεδομένα του ως ένα dictionary. Εάν το αρχείο δεν βρεθεί, εμφανίζει ένα κατάλληλο μήνυμα σφάλματος και επιστρέφει ένα κενό dictionary.

In [13]:
def loadIndex():
   
    inputFile = "inverted_index.json"
    try:
        with open(inputFile, "r", encoding="utf-8") as file:
            inverted_index = json.load(file)
        print(f"Φορτώθηκε το ευρετήριο από το αρχείο: {inputFile}")
        return inverted_index
    except FileNotFoundError:
        print("Το αρχείο ευρετηρίου δεν βρέθηκε.")
        return {}

Η συνάρτηση queryValidation ελέγχει την εγκυρότητα ενός Boolean ερωτήματος. Συγκεκριμένα, εξετάζει εάν υπάρχει ακολουθία διαδοχικών τελεστών που δεν επιτρέπεται. Εάν το ερώτημα δεν πληροί τους παρακάτω κανόνες επιστρέφει False , αλλιώς επιστρέφει True.

In [14]:
def queryValidation(terms):

    logicalOp = {'and', 'or', 'not'}
    
    if terms[0].lower() in {'and', 'or'}:
        return False, "Το ερώτημα δεν μπορεί να ξεκινά με AND ή OR"
    
    if terms[-1].lower() in logicalOp:
        return False, "Το ερώτημα δεν μπορεί να τελειώνει με τελεστή"
    
    for i in range(len(terms) - 1):
        if terms[i].lower() in logicalOp and terms[i+1].lower() in logicalOp - {'not'}:
            return False, "Μη έγκυρη ακολουθία τελεστών"
        
    for i in range(len(terms) - 1):
        if terms[i].lower() == 'not' and terms[i+1].lower() in logicalOp:
            return False, "Το NOT δεν μπορεί να ακολουθείται από τελεστή"
        
    return True, ""


Η συνάρτηση processQuery επεξεργάζεται Boolean queries για την ανάκτηση εγγράφων από το αντεστραμμένο ευρετήριο. Αναλύει το query, εφαρμόζει τους τελεστές AND, OR, NOT και επιστρέφει τα σχετικά έγγραφα. Εάν το ερώτημα είναι άκυρο, επιστρέφει κενό σύνολο.

In [15]:
def processTerm(term, docList, inverted_index, finalResult, logicalOp=None):

    documents = set(inverted_index.get(term, {}).get("documents", []))
    
    if logicalOp == "not":
        documents = docList.difference(documents)
    if finalResult is None:
        return documents
    elif logicalOp == "and":
        return finalResult.intersection(documents)
    elif logicalOp == "or":
        return finalResult.union(documents)
    else: 
        return finalResult.intersection(documents)

def processQuery(query, inverted_index):
    terms = query.lower().split()
    valid, errorMess = queryValidation(terms)
    if not valid:
        print(f"Σφάλμα: {errorMess}")
        return set()
    
    print(f"Αρχικό ερώτημα: {query}")
    print(f"Αναζητούμενες λέξεις: {', '.join([term for term in terms if term not in ['and', 'or', 'not']])}")

    docList = set()
    for docs in inverted_index.values():
        docList.update(docs.get("documents", []))

    results = None
    i = 0

    while i < len(terms):
        term = terms[i]

        if term == "not":
            i += 1
            if i < len(terms):
                results = processTerm(terms[i], docList, inverted_index, results, logicalOp="not")
            i += 1

        elif term == "and":
            i += 1
            if i < len(terms):
                if terms[i] == "not":
                    i += 1
                    if i < len(terms):
                        results = processTerm(terms[i], docList, inverted_index, results, logicalOp="not")
                else:
                    results = processTerm(terms[i], docList, inverted_index, results, logicalOp="and")
            i += 1

        elif term == "or":
            i += 1
            if i < len(terms):
                if terms[i] == "not":
                    i += 1
                    if i < len(terms):
                        results = processTerm(terms[i], docList, inverted_index, results, logicalOp="not")
                else:
                    results = processTerm(terms[i], docList, inverted_index, results, logicalOp="or")
            i += 1

        else:
            results = processTerm(term, docList, inverted_index, results)
            i += 1

    return results if results is not None else set()


#------------Παράδειγμα εκτέλεσης συνάρτησης-------------#

def testQueries():
    queries = [
        "machine learning",
        "deep neural",
        "artificial intelligence",
        "data mining",
        "machine and learning",
        "deep or neural",
        "machine and not deep",
        "neural networks",
        "supervised learning",
        "reinforcement learning"
    ]
    
    for query in queries:
        with open("inverted_index.json", "r", encoding="utf-8") as file:
            inverted_index = json.load(file)
        results = processQuery(query, inverted_index)
        print(f"Έγγραφα: {sorted(list(results))}\n")

testQueries()

Αρχικό ερώτημα: machine learning
Αναζητούμενες λέξεις: machine, learning
Έγγραφα: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Αρχικό ερώτημα: deep neural
Αναζητούμενες λέξεις: deep, neural
Έγγραφα: [1, 2, 3, 4, 7, 8, 10, 11, 12, 13, 15]

Αρχικό ερώτημα: artificial intelligence
Αναζητούμενες λέξεις: artificial, intelligence
Έγγραφα: [1, 2, 3, 10, 12, 13]

Αρχικό ερώτημα: data mining
Αναζητούμενες λέξεις: data, mining
Έγγραφα: [1, 12, 13]

Αρχικό ερώτημα: machine and learning
Αναζητούμενες λέξεις: machine, learning
Έγγραφα: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Αρχικό ερώτημα: deep or neural
Αναζητούμενες λέξεις: deep, neural
Έγγραφα: [1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 13, 15]

Αρχικό ερώτημα: machine and not deep
Αναζητούμενες λέξεις: machine, deep
Έγγραφα: [5, 6, 9, 14]

Αρχικό ερώτημα: neural networks
Αναζητούμενες λέξεις: neural, networks
Έγγραφα: []

Αρχικό ερώτημα: supervised learning
Αναζητούμενες λέξεις: supervised, learning
Έγγραφα: [1, 2, 3, 5, 6, 7, 8, 1

ΕΡΩΤΗΜΑ Β


Η συνάρτηση prepareDocuments επεξεργάζεται το αντεστραμμένο ευρετήριο για να δημιουργήσει το περιεχόμενο των εγγράφων σε κατάλληλη μορφή για τον υπολογισμό του TF-IDF. Ομαδοποιεί τους όρους κάθε εγγράφου, τους ενώνει σε μία συμβολοσειρά και επιστρέφει λίστες με τα κείμενα των εγγράφων και τα αντίστοιχα IDs.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def prepareDocuments(inverted_index):
    documents = {}
    
    for term, data in inverted_index.items():
        for docId in data.get("documents", []):
            if docId not in documents:
                documents[docId] = []

            Freq = data["frequency"].get(str(docId), 0)  
            documents[docId].extend([term] * Freq)
    
    documentTexts = [" ".join(terms) for docId, terms in sorted(documents.items())]
    documentIds = [docId for docId, _ in sorted(documents.items())]
    
    return documentTexts, documentIds

Η συνάρτηση computeTfidf υπολογίζει το TF-IDF για μια λίστα εγγράφων. Επιστρέφει τον πίνακα TF-IDF, τα χαρακτηριστικά και το ίδιο το αντικείμενο vectorizer.

In [17]:
def computeTfidf(documents):
    
    vectorizer = TfidfVectorizer()
    tableTfidf = vectorizer.fit_transform(documents)
    newNames = vectorizer.get_feature_names_out()
    return tableTfidf, newNames, vectorizer

Η συνάρτηση rank_documents κατατάσσει έγγραφα με βάση τη συνάφεια τους με ένα ερώτημα. Μετατρέπει το ερώτημα σε διάνυσμα TF-IDF, υπολογίζει το cosine similarity με τα έγγραφα και τα επιστρέφει ταξινομημένα κατά βαθμολογία.

In [18]:
def get_score(item):
    return item[1]

def rankDocuments(query, TableTfidf, vectorizer, doc_ids):

    tfidfQuery = vectorizer.transform([query])
    cos = cosine_similarity(tfidfQuery, TableTfidf).flatten()
    documentScores = [(doc_id, score) for doc_id, score in zip(doc_ids, cos)]
    rankedDocuments = sorted(documentScores, key=get_score, reverse=True)
    
    return rankedDocuments

Εγκατάσταση της rank-bm25

In [19]:
!pip install rank-bm25



Η συνάρτηση rankDocuments_bm25 βρίσκει ποια έγγραφα ταιριάζουν περισσότερο με ένα ερώτημα, χρησιμοποιώντας τον αλγόριθμο BM25. Χωρίζει τα έγγραφα και το ερώτημα σε λέξεις, υπολογίζει βαθμολογίες για κάθε έγγραφο και επιστρέφει τα έγγραφα ταξινομημένα.

In [20]:
from rank_bm25 import BM25Okapi

def rankDocuments_bm25(query, documents):
 
    tokenizedDoc = [doc.split() for doc in documents]
    bm25 = BM25Okapi(tokenizedDoc)
    tokenizedQuery = query.split()
    scores = bm25.get_scores(tokenizedQuery)
    rankedResults = sorted(enumerate(scores), key=get_score, reverse=True)

    return rankedResults


Η συνάρτηση computeVsm μετατρέπει τα κείμενα και το ερώτημα σε διανύσματα χρησιμοποιώντας το CountVectorizer, κανονικοποιεί τα διανύσματα, και υπολογίζει την ομοιότητα συνημίτονου μεταξύ του ερωτήματος και των κειμένων για να επιστρέψει τα έγγραφα ταξινομημένα με βάση τη σχετικότητά τους.

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

def computeVsm(query, documents, document_ids):
    vectorizer = CountVectorizer()
    tablesDoc = vectorizer.fit_transform(documents)
    tableQuery = vectorizer.transform([query])
    normalizedDocs = normalize(tablesDoc)
    normalizedQuery = normalize(tableQuery)
    similarities = (normalizedQuery @ normalizedDocs.T).toarray().flatten()
    rankedResults = sorted(zip(document_ids, similarities), key=lambda x: x[1], reverse=True)
    
    return rankedResults 

Η main φορτώνει το ευρετήριο και τα έγγραφα, και στη συνέχεια προσφέρει ένα μενού στον χρήστη για να μπορεί να επιλέξει ανάμεσα στους διαφορετικούς αλγορίθμους που έχουμε υλοποιήσει. Ο χρήστης μπορεί να εισάγει ένα ερώτημα και να λάβει τα αντίστοιχα αποτελέσματα ταξινομημένα με βάση τη σχετικότητα.

In [22]:
def menu():

    invertedIndex = loadIndex()
    docstexts, docIds = prepareDocuments(invertedIndex)

    while True:
        print("\nΕπιλέξτε αλγόριθμο αναζήτησης:")
        print("1: Boolean Retrieval")
        print("2: TF-IDF Ranking")
        print("3: BM25 Ranking")
        print("4: Vector Space Model (VSM)")
        print("0: Έξοδος")
        choice = input("Επιλογή: ")

        if choice == "0":
            print("Τερματισμός προγράμματος.")
            break

        query = input("\nΕισάγετε το ερώτημά σας: ")

        if choice == "1":
            results = processQuery(query, invertedIndex)
            print("\nΑποτελέσματα Boolean Retrieval:")
            print(sorted(results))

        elif choice == "2":
            tableTfidf, newNames, vectorizer = computeTfidf(docstexts)
            rankedResults = rankDocuments(query, tableTfidf, vectorizer, docIds)
            print("\nΑποτελέσματα TF-IDF Ranking:")
            for id, score in rankedResults[:10]:
                if score > 0:
                    print(f"Έγγραφο: {id:<20} Βαθμολογία: {score:.4f}")

        elif choice == "3":
            rankedResults = rankDocuments_bm25(query, docstexts)
            print("\nΑποτελέσματα BM25 Ranking:")
            for dx, score in rankedResults[:10]:
                if score > 0:
                    print(f"Έγγραφο: {docIds[dx]:<20} Βαθμολογία: {score:.4f}")

        elif choice == "4":
            rankedResults = computeVsm(query, docstexts, docIds)
            print("\nΑποτελέσματα Vector Space Model (VSM):")
            for id, score in rankedResults[:10]:
                if score > 0:
                    print(f"Έγγραφο: {id:<20} Βαθμολογία: {score:.4f}")

        else:
            print("Μη έγκυρη επιλογή. Προσπαθήστε ξανά.")

menu()

Φορτώθηκε το ευρετήριο από το αρχείο: inverted_index.json

Επιλέξτε αλγόριθμο αναζήτησης:
1: Boolean Retrieval
2: TF-IDF Ranking
3: BM25 Ranking
4: Vector Space Model (VSM)
0: Έξοδος
Αρχικό ερώτημα: machine learning
Αναζητούμενες λέξεις: machine, learning

Αποτελέσματα Boolean Retrieval:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

Επιλέξτε αλγόριθμο αναζήτησης:
1: Boolean Retrieval
2: TF-IDF Ranking
3: BM25 Ranking
4: Vector Space Model (VSM)
0: Έξοδος
Μη έγκυρη επιλογή. Προσπαθήστε ξανά.

Επιλέξτε αλγόριθμο αναζήτησης:
1: Boolean Retrieval
2: TF-IDF Ranking
3: BM25 Ranking
4: Vector Space Model (VSM)
0: Έξοδος

Αποτελέσματα BM25 Ranking:
Έγγραφο: 1                    Βαθμολογία: 1.8696
Έγγραφο: 10                   Βαθμολογία: 1.8572
Έγγραφο: 2                    Βαθμολογία: 1.8537
Έγγραφο: 8                    Βαθμολογία: 1.8303
Έγγραφο: 5                    Βαθμολογία: 1.8081
Έγγραφο: 12                   Βαθμολογία: 1.8021
Έγγραφο: 13                   Βαθμολογία: 1.8021
Έ

ΒΗΜΑ 5

In [24]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, average_precision_score

queries = [
    "machine learning",
    "artificial intelligence", 
    "statistical analysis",
    "natural language processing",
    "computer vision",
    "speech recognition", 
    "predictive analytics",
    "data mining",
    "unsupervised learning",
    "algorithm optimization",
    "mathematical model",
    "machine learning AND artificial intelligence",
    "deep learning NOT neural networks",
    "statistical OR mathematical",
    "computer vision AND deep learning",
    "model performance",
    "artificial intelligence method",
    "machine learning AND statistical NOT deep",
    "neural network AND optimization"
]

results = [
    [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15],
    [1,2,3,9,10,15],
    [1,3,4,9,10,13,14],
    [1,2,3,5,7,9,10],
    [1,3,4,5,7,9,10,11],
    [1,3,4,7,9,10,14],
    [1,9,10],
    [1,9,10],
    [1,2,3,9,10,11,13,14],
    [1,2,3,4,7,9,10,11,12,13,15],
    [1,2,9,10],
    [1,2,3,9,10,15],
    [],
    [1,2,3,4,7,9,10,13,14],
    [1,3,4,5,7,9,10],
    [1,2,3,7,9,10,11,12,13,14,15],
    [1,2,3,9,10,15],
    [13],
    [1,2,3,4,7,9,10,11,12,15]
]

def binaryMatrix():
    matrix = np.zeros((len(queries), 15))
    for i, result in enumerate(results):
        for doc_id in result:
            matrix[i][doc_id-1] = 1
    return matrix

def calculateMetrics(matrix):
    query_metrics = []
    
    for i, query in enumerate(queries):
        relevant_docs = np.where(matrix[i] == 1)[0]
        total_docs = len(matrix[i])
        retrieved_docs = len(results[i])
        
        if retrieved_docs == 0:
            precision = 0
            recall = 0
            f1 = 0
        else:
            precision = len(relevant_docs) / total_docs
            recall = len(relevant_docs) / retrieved_docs
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
        query_metrics.append({
            'query': query,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'retrieved_docs': retrieved_docs,
            'relevant_docs': len(relevant_docs)
        })
    
    return query_metrics

def printResults(matrix, query_metrics):
    print("\nΔυαδικός πίνακας εγγράφων:")
    print("Έγγραφα:  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15")
    print("-" * 50)
    for i, query in enumerate(queries):
        row = ' '.join(f'{int(x):2}' for x in matrix[i])
        print(f"{query[:20]:<20} {row}")
    

    for m in query_metrics:
        print(f"\nΕρώτημα: {m['query']}")
        print(f"Ακρίβεια: {m['precision']:.3f}")
        print(f"Ανάκληση: {m['recall']:.3f}")
        print(f"F1 Score: {m['f1']:.3f}")
        print(f"Ανακτημένα έγγραφα: {m['retrieved_docs']}")
        print(f"Σχετικά έγγραφα: {m['relevant_docs']}")
    
    avg_precision = np.mean([m['precision'] for m in query_metrics])
    avg_recall = np.mean([m['recall'] for m in query_metrics])
    avg_f1 = np.mean([m['f1'] for m in query_metrics])
    
    print("\nΣυνολική απόοδση συστήματος:")
    print("-" * 30)
    print(f"Μέση ακρίβεια: {avg_precision:.3f}")
    print(f"Μέση ανάκληση: {avg_recall:.3f}")
    print(f"Μέσο F1 Score: {avg_f1:.3f}")

if __name__ == "__main__":

    relevance_matrix = binaryMatrix()
    query_metrics = calculateMetrics(relevance_matrix)
    printResults(relevance_matrix, query_metrics)


Δυαδικός πίνακας εγγράφων:
Έγγραφα:  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
--------------------------------------------------
machine learning      1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
artificial intellige  1  1  1  0  0  0  0  0  1  1  0  0  0  0  1
statistical analysis  1  0  1  1  0  0  0  0  1  1  0  0  1  1  0
natural language pro  1  1  1  0  1  0  1  0  1  1  0  0  0  0  0
computer vision       1  0  1  1  1  0  1  0  1  1  1  0  0  0  0
speech recognition    1  0  1  1  0  0  1  0  1  1  0  0  0  1  0
predictive analytics  1  0  0  0  0  0  0  0  1  1  0  0  0  0  0
data mining           1  0  0  0  0  0  0  0  1  1  0  0  0  0  0
unsupervised learnin  1  1  1  0  0  0  0  0  1  1  1  0  1  1  0
algorithm optimizati  1  1  1  1  0  0  1  0  1  1  1  1  1  0  1
mathematical model    1  1  0  0  0  0  0  0  1  1  0  0  0  0  0
machine learning AND  1  1  1  0  0  0  0  0  1  1  0  0  0  0  1
deep learning NOT ne  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
statistic